In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *
import importlib, pipeline_helper

In [ ]:
importlib.reload(pipeline_helper)
from pipeline_helper import allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms

In [ ]:
import numpy as np
import numpy.linalg as la

In [ ]:
import pickle, gzip

### Initialization

In [ ]:
name = 'lilium_top'
input_path = '../../data/{}.json.gz'.format(name)

In [ ]:
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
import mesh
target_mesh = mesh.Mesh(input_data['target_v'], input_data['target_f'])

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
thickness = io.material_params[6]
target_height_multiplier = input_data['target_spacing_factor']
# target_height_multiplier = 1

In [ ]:
# curr_um = pickle.load(gzip.open('../../output/lilium_top_optimized_equilibrium_2022_01_19_16_45_target_height_factor_5.0.pkl.gz', 'r'))

#### Pin Rigid Motion



In [ ]:
use_pin = False

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

In [ ]:
rod_colors = get_color_field(curr_um, input_data)

In [ ]:
# lview = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# lview.update(scalarField = rod_colors)
# lview.show()

In [ ]:
import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
view.getCameraParams()

In [ ]:
view.getSize()

In [ ]:
allEnergies(curr_um)

In [ ]:
import time

In [ ]:
# def renderToFile(path, view, renderCam = None):
#     orender = view.offscreenRenderer(width=2048, height=1200)
#     if renderCam is not None:
#         orender.setCameraParams(renderCam)
#     orender.render()
#     orender.save(path)
# renderToFile('{}_target_height_factor_{}_{}.png'.format(name, target_height_multiplier, time.strftime("%Y_%m_%d_%H_%M")), view)

In [ ]:
curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.LinearActuator

In [ ]:
import importlib, pipeline_helper
importlib.reload(pipeline_helper)

In [ ]:
from pipeline_helper import set_joint_vector_field, show_center_joint_normal, show_joint_normal

In [ ]:
from equilibrium_solve_analysis import EquilibriumSolveAnalysis
eqays = EquilibriumSolveAnalysis(curr_um)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
curr_um.targetDeploymentHeight = thickness * target_height_multiplier
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 1e-1
curr_um.setHoldClosestPointsFixed(True)
curr_um.scaleInputPosWeights(1)

In [ ]:
curr_um.angleBoundEnforcement = umbrella_mesh.AngleBoundEnforcement.Penalty

In [ ]:
allGradientNorms(curr_um)

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    # if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
    dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [ ]:
view.showScalarField(rod_colors)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-3
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e-2
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e-1
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e0
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e0
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
eqays.plot()

### Initialize Design Optimization

In [ ]:
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 1e-3
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.1)

In [ ]:
import umbrella_optimization
import umbrella_optimization_finite_diff

In [ ]:
import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = opt_opts, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
opt_opts.niter = 50

In [ ]:
results.success

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

In [ ]:
optimizer.beta = 1 * 1e6
optimizer.gamma = 1
optimizer.eta = 0
optimizer.zeta = 0# 1e1

In [ ]:
rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
four_parameters_optimizer = umbrella_optimization.UmbrellaFourParametersOptimization(optimizer)
single_rest_height_optimizer = umbrella_optimization.UmbrellaSingleRestHeightOptimization(rest_height_optimizer)

In [ ]:
original_design_parameters = four_parameters_optimizer.params()

In [ ]:
import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

In [ ]:
from pipeline_helper import UmbrellaOptimizationCallback

In [ ]:
from umbrella_optimization import OptEnergyType

In [ ]:
from design_optimization_analysis import DesignOptimizationAnalysis
doptays = DesignOptimizationAnalysis(four_parameters_optimizer)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
import time

In [ ]:
pipeline_helper.prev_time_stamp = time.time()

In [ ]:
uo = four_parameters_optimizer.get_parent_opt()

In [ ]:
uo.equilibriumOptimizer.options.verbose = 1
#uo.equilibriumOptimizer.options.verboseWorkingSet = True
uo.equilibriumOptimizer.options.gradTol = 1e-10
# Hold the closest points fixed in the target-attraction term of the equilibrium solve:
# this seems to make the design optimization much more robust.
uo.setHoldClosestPointsFixed(True, False)

In [ ]:
tfview = pipeline_helper.TargetFittingVisualization(curr_um, uo.target_surface_fitter, view)
cb = pipeline_helper.UmbrellaOptimizationCallback(four_parameters_optimizer, view, True, False, 1, rod_colors, doptays.record, tfview=tfview)

In [ ]:
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
solverStatus = umbrella_optimization.optimize(four_parameters_optimizer, algorithm, 50, 0.005, 1e-5, cb, input_data["plate_edge_length"] / 30 * 32)

In [ ]:
four_parameters_optimizer.reset_joint_target_with_closest_points()

In [ ]:
four_parameters_optimizer.beta = 1 * 1e7

In [ ]:
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
solverStatus = umbrella_optimization.optimize(four_parameters_optimizer, algorithm, 10000, 0.005, 1e-5, cb, input_data["plate_edge_length"] / 30 * 32)

In [ ]:
four_parameters_optimizer.beta, four_parameters_optimizer.gamma, four_parameters_optimizer.eta

In [ ]:
import time

In [ ]:
doptays.plot()

### Get true equilibrium state

In [ ]:
use_pin = True

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-1
curr_um.uniformDeploymentEnergyWeight = 0
curr_um.targetDeploymentHeight = thickness * target_height_multiplier * 5
curr_um.attractionWeight = 0

In [ ]:
allEnergies(curr_um)

In [ ]:
OPTS.niter = 500

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
import pickle 
import gzip

pickle.dump(curr_um, gzip.open('../../output/{}_four_parameters_optimized_rest_state_equilibrium_{}_target_height_factor_{}.pkl.gz'.format(name, time.strftime("%Y_%m_%d_%H_%M"), target_height_multiplier), 'w'))
# load_um = pickle.load(gzip.open('test_pickle_um.pkl.gz', 'r'))

In [ ]:
import pickle 
import gzip

pickle.dump(curr_um, gzip.open('../../output/{}_four_parameters_optimized_equilibrium_{}_target_height_factor_{}.pkl.gz'.format(name, time.strftime("%Y_%m_%d_%H_%M"), target_height_multiplier), 'w'))
# load_um = pickle.load(gzip.open('test_pickle_um.pkl.gz', 'r'))

In [ ]:
from load_jsondata import update_optimized_json
update_optimized_json(input_path, rest_height_optimizer.params(), output_json_path = '../../output/{}_four_parameters_optimized_params_{}.json'.format(name, time.strftime("%Y_%m_%d_%H_%M")), optim_spacing_factor = target_height_multiplier)